In [ ]:
!pip install --upgrade tensorflow
!pip install autokeras

In [1]:
import autokeras as ak
from tensorflow.keras.datasets import mnist
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Since the expected dataset is of the format expected by generator, gen used and converted to array for simplicity sake.
def _load_dataset_into_gen(dataset_location):
    filename = os.path.basename(dataset_location).split('.')[0]
    if(dataset_location.endswith(".zip")):
        extraction_dir = os.path.splitext(dataset_location)[0]
        if (not os.path.isdir(extraction_dir)):
            try:
                with zipfile.ZipFile(dataset_location,"r") as zip_ref:
                    zip_ref.extractall(path=extraction_dir)
            except Exception as e:
                print(e)
        dataset_location = os.path.join(extraction_dir, filename)
    training_data_location = os.path.join(dataset_location, "train")
    test_data_location = os.path.join(dataset_location, "test")
    if not os.path.exists(training_data_location):
        print(f"Unable to find training data directory at location {training_data_location}")
        return
    if not os.path.exists(test_data_location):
        print(f"Unable to find training data directory at location {test_data_location}")
        return

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
    train_generator = train_datagen.flow_from_directory(training_data_location,class_mode='sparse', shuffle=True, target_size=(128,128), seed=42)

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
    test_generator = test_datagen.flow_from_directory(test_data_location, class_mode='sparse',target_size=(128,128), shuffle=True, seed=42)
    return train_generator, test_generator

def _get_arr_from_gen(generator):
    x_size = (generator.samples,) + (generator.image_shape)
    y_size = (generator.samples,) + (generator.num_classes,) 
    x_app = np.empty(shape = x_size)
    y_app = []

    batch_index = 0
    batch_size = generator.batch_size
    while batch_index <= generator.batch_index:
        data = generator.next()
        for idx, x in enumerate(data[0]):
            x_app[(idx + (batch_index*batch_size))] = x
        for _, x in enumerate(data[1]):
            y_app.append(x)
        batch_index = batch_index + 1

    return x_app, np.array(y_app)

In [3]:
dataset_location = "./Data/tiny2"
train_gen, test_gen = _load_dataset_into_gen(dataset_location)
(x_train, y_train) = _get_arr_from_gen(train_gen)
(x_test, y_test) = _get_arr_from_gen(test_gen)

Found 30 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [4]:
clf = ak.ImageClassifier(max_trials=5)
history = clf.fit(x_train, y_train, epochs = 3)

Train for 1 steps, validate for 1 steps
Epoch 1/3
1/1 [==============================] - 1s 896ms/step - loss: 0.6805 - accuracy: 0.7083 - val_loss: 1.7858 - val_accuracy: 0.3333
Epoch 2/3
1/1 [==============================] - 0s 493ms/step - loss: 1.1315 - accuracy: 0.5417 - val_loss: 2.0895 - val_accuracy: 0.6667
Epoch 3/3
1/1 [==============================] - 1s 525ms/step - loss: 2.6421 - accuracy: 0.4583 - val_loss: 1.3455 - val_accuracy: 0.6667


Train for 1 steps, validate for 1 steps
Epoch 1/3
1/1 [==============================] - 7s 7s/step - loss: 0.6704 - accuracy: 0.5000 - val_loss: 0.7881 - val_accuracy: 0.3333
Epoch 2/3
1/1 [==============================] - 1s 1s/step - loss: 0.5984 - accuracy: 0.6667 - val_loss: 0.8666 - val_accuracy: 0.6667
Epoch 3/3
1/1 [==============================] - 2s 2s/step - loss: 3.7093 - accuracy: 0.5000 - val_loss: 0.6795 - val_accuracy: 0.5000


Train for 1 steps, validate for 1 steps
Epoch 1/3
1/1 [==============================] - 7s 7s/step - loss: 0.6961 - accuracy: 0.5000 - val_loss: 0.9511 - val_accuracy: 0.3333
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 1.2404 - accuracy: 0.5000 - val_loss: 0.6714 - val_accuracy: 0.6667
Epoch 3/3
1/1 [==============================] - 2s 2s/step - loss: 1.9273 - accuracy: 0.5833 - val_loss: 0.7609 - val_accuracy: 0.3333


INFO:tensorflow:Oracle triggered exit
Train for 2 steps, validate for 1 steps
Epoch 1/3
2/2 [==============================] - ETA: 5s - loss: 0.6423 - accuracy: 0.58 - 6s 3s/step - loss: 0.9674 - accuracy: 0.5667 - val_loss: 0.6650 - val_accuracy: 0.6667
Epoch 2/3
2/2 [==============================] - ETA: 1s - loss: 1.4998 - accuracy: 0.45 - 2s 960ms/step - loss: 1.0212 - accuracy: 0.5333 - val_loss: 0.9650 - val_accuracy: 0.3333
Epoch 3/3
2/2 [==============================] - ETA: 1s - loss: 1.2953 - accuracy: 0.58 - 2s 953ms/step - loss: 0.8008 - accuracy: 0.6667 - val_loss: 0.7098 - val_accuracy: 0.6667


In [7]:
predicted_y = clf.predict(x_test)
print(predicted_y)

[[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [8]:
print(clf.evaluate(x_test, y_test))

1/1 [==============================] - 1s 920ms/step - loss: 0.7722 - accuracy: 0.5000
[0.7722382545471191, 0.5]


In [9]:
model = clf.export_model()

In [10]:
model.save("out")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: out/assets


In [23]:
model = tf.keras.models.load_model("out")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 128, 128, 3)  7           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           normalization[0][0]              
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
______________________________________________________________________________________________

conv2_block2_1_relu (Activation (None, 32, 32, 64)   0           conv2_block2_1_bn[0][0]          
__________________________________________________________________________________________________
conv2_block2_2_pad (ZeroPadding (None, 34, 34, 64)   0           conv2_block2_1_relu[0][0]        
__________________________________________________________________________________________________
conv2_block2_2_conv (Conv2D)    (None, 32, 32, 64)   36864       conv2_block2_2_pad[0][0]         
__________________________________________________________________________________________________
conv2_block2_2_bn (BatchNormali (None, 32, 32, 64)   256         conv2_block2_2_conv[0][0]        
__________________________________________________________________________________________________
conv2_block2_2_relu (Activation (None, 32, 32, 64)   0           conv2_block2_2_bn[0][0]          
__________________________________________________________________________________________________
conv2_bloc

__________________________________________________________________________________________________
conv3_block2_3_conv (Conv2D)    (None, 16, 16, 512)  66048       conv3_block2_2_relu[0][0]        
__________________________________________________________________________________________________
conv3_block2_out (Add)          (None, 16, 16, 512)  0           conv3_block1_out[0][0]           
                                                                 conv3_block2_3_conv[0][0]        
__________________________________________________________________________________________________
conv3_block3_preact_bn (BatchNo (None, 16, 16, 512)  2048        conv3_block2_out[0][0]           
__________________________________________________________________________________________________
conv3_block3_preact_relu (Activ (None, 16, 16, 512)  0           conv3_block3_preact_bn[0][0]     
__________________________________________________________________________________________________
conv3_bloc

__________________________________________________________________________________________________
conv4_block2_1_conv (Conv2D)    (None, 8, 8, 256)    262144      conv4_block2_preact_relu[0][0]   
__________________________________________________________________________________________________
conv4_block2_1_bn (BatchNormali (None, 8, 8, 256)    1024        conv4_block2_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_1_relu (Activation (None, 8, 8, 256)    0           conv4_block2_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block2_2_pad (ZeroPadding (None, 10, 10, 256)  0           conv4_block2_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block2_2_conv (Conv2D)    (None, 8, 8, 256)    589824      conv4_block2_2_pad[0][0]         
__________

conv4_block5_2_relu (Activation (None, 8, 8, 256)    0           conv4_block5_2_bn[0][0]          
__________________________________________________________________________________________________
conv4_block5_3_conv (Conv2D)    (None, 8, 8, 1024)   263168      conv4_block5_2_relu[0][0]        
__________________________________________________________________________________________________
conv4_block5_out (Add)          (None, 8, 8, 1024)   0           conv4_block4_out[0][0]           
                                                                 conv4_block5_3_conv[0][0]        
__________________________________________________________________________________________________
conv4_block6_preact_bn (BatchNo (None, 8, 8, 1024)   4096        conv4_block5_out[0][0]           
__________________________________________________________________________________________________
conv4_block6_preact_relu (Activ (None, 8, 8, 1024)   0           conv4_block6_preact_bn[0][0]     
__________

conv5_block3_preact_relu (Activ (None, 4, 4, 2048)   0           conv5_block3_preact_bn[0][0]     
__________________________________________________________________________________________________
conv5_block3_1_conv (Conv2D)    (None, 4, 4, 512)    1048576     conv5_block3_preact_relu[0][0]   
__________________________________________________________________________________________________
conv5_block3_1_bn (BatchNormali (None, 4, 4, 512)    2048        conv5_block3_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block3_1_relu (Activation (None, 4, 4, 512)    0           conv5_block3_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_2_pad (ZeroPadding (None, 6, 6, 512)    0           conv5_block3_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_bloc

In [17]:
# model.compile(loss='binary_crossentropy',
#                         optimizer='rmsprop',
#                         metrics=['accuracy'])

In [18]:
model

In [29]:
modelpredi

20/20 [==============================] - 0s 10ms/sample - loss: 0.7722 - accuracy: 0.5000


[0.77223801612854, 0.5]